In [8]:
import pandas as pd
import glob
import scipy.io as io
import numpy
from tfcat import TFCat
from shapely.geometry import MultiPoint, Point, Polygon, MultiPolygon
from astropy.time import Time
import matplotlib.pyplot as pyplot
import matplotlib.colors as colors
import h5py
from h5py import File
import numpy as np
from datetime import datetime
from os import path
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import csv
from tqdm import tqdm
import seaborn as sns 
from scipy.stats import binned_statistic_2d
from shapely.ops import unary_union
from hdf5_routines import data_to_hdf5
from polar_masked_data import get_polygons, compute_data_points, compute_mask, apply


plt.rcParams["figure.figsize"] = (200,100)


In [15]:
#path to catalogue, data and ephemeris:


dir_save_name = "/Users/clouis/Documents/Data/POLAR/data"
path_data=glob.glob("/Users/clouis/Documents/Data/POLAR/data/po_h1_pwi.preprocessed.hdf5")[0]
path_cat=glob.glob("/Users/clouis/Documents/Data/POLAR/data/catalogue_Polar.json")[0]

#time_view_start = '1996-03-25'
#time_view_end ='1996-05-03'
time_view_start = '1996-01-01'
time_view_end ='1996-07-01'
val='Flux'
type_=['AKR','AKR1','AKR2','HAKR','HAKR1','HAKR2','FCE','FCE1','FCE2']
#type_=['HAKR','HAKR1','HAKR2']
#type_=['LFE','LFE1', 'LFE2']
#type_=['FCE','FCE1', 'FCE2']
#type_=['AKR','AKR1', 'AKR2']


In [16]:
# load data
file_data = h5py.File(path_data, 'r')
cols = list(file_data.keys())

date_data = numpy.array(Time(file_data.get(cols[3])[:], format = "jd").unix)
flux = numpy.array(file_data.get(cols[1])[:], dtype="float")
frequencies = numpy.array(file_data.get(cols[2])[:])





In [17]:
flux = flux[(date_data >= Time(time_view_start).unix) & (date_data <= Time(time_view_end).unix),:]
date_data = date_data[(date_data >= Time(time_view_start).unix) & (date_data < Time(time_view_end).unix)]



In [18]:
# sorted data to avoid issue later

sorted_time, sorted_data = zip(*sorted(zip(date_data, flux)))
del date_data, flux
date_data, flux = numpy.array(sorted_time), numpy.array(sorted_data)

In [19]:
#Importing polygons 
#polygon_array=get_polygons(path_cat, time_view_start, time_view_end, type_)
print(type_)
cat = TFCat.from_file(path_cat)
cat.feature(0).geometry.type

event = cat.feature(0)
crs = cat.crs
tmin = crs.time_converter(event.bbox[0]).isot
tmax = crs.time_converter(event.bbox[2]).isot

polygon_array=get_polygons(path_cat, time_view_start, time_view_end, type_)

['AKR', 'AKR1', 'AKR2', 'HAKR', 'HAKR1', 'HAKR2', 'FCE', 'FCE1', 'FCE2']


In [23]:
polygon_array['AKR'].geometry

AttributeError: 'MultiPolygon' object has no attribute 'geometry'

In [12]:
# Preparing data for the filtering
#coords = [(t, f) for t,f in zip(date_data.flatten(), frequencies.flatten())]
#data_len = len(flux)
#index = numpy.arange(data_len, dtype=int)
#data_points = MultiPoint([Point(x, y, z) for (x, y), z in zip(coords, index)])
#polygon = Polygon(event.geometry.coordinates[0])

data_points = compute_data_points(flux, date_data, frequencies)




In [35]:
mask = compute_mask(flux, date_data, frequencies, polygon_array, type_, data_points)

  0%|          | 0/3 [00:00<?, ?it/s]

AKR


 33%|███▎      | 1/3 [08:34<17:09, 514.87s/it]

AKR1


 67%|██████▋   | 2/3 [09:01<03:47, 227.67s/it]

AKR2


100%|██████████| 3/3 [09:10<00:00, 183.40s/it]


In [37]:
masked_data = apply(flux, mask, type_, invert = False)

AKR
AKR1
AKR2


In [38]:
print(dir_save_name)
time = date_data
frequency = frequencies
if not os.path.exists(dir_save_name):
    os.makedirs(dir_save_name)
data_to_hdf5(dir_save_name + '/polar_AKR_mask.hdf5'  , time, frequencies, masked_data)

/Users/clouis/Documents/Data/POLAR/data
